In [ ]:
import pandas as pd
import numpy as np
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

**IMPORTING DATASETS**

In [ ]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata
!chmod 600 /content/kaggle.json

100% 8.89M/8.89M [00:00<00:00, 93.1MB/s]
100% 8.89M/8.89M [00:00<00:00, 92.3MB/s]


In [ ]:
!unzip \*.zip && rm *.zip

Archive:  tmdb-movie-metadata.zip
replace tmdb_5000_credits.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: tmdb_5000_credits.csv   
replace tmdb_5000_movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: tmdb_5000_movies.csv    


In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


**MERGING TWO DATASETS BASED ON TITLE**

In [ ]:
df = movies.merge(credits,on = 'title')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

**FEATURE EXTRACTION**

In [ ]:
movie = df[['genres','keywords','title','overview','movie_id','cast','crew']]

In [ ]:
movie.isnull().sum()

genres      0
keywords    0
title       0
overview    3
movie_id    0
cast        0
crew        0
dtype: int64

In [ ]:
movie.dropna(inplace = True)

<ipython-input-43-d2f34635faad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie.dropna(inplace = True)


In [ ]:
movie.duplicated().sum()

0

In [ ]:
movie.iloc[0]['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
import ast
def convert(obj):
  l = []
  for i in ast.literal_eval(obj):   #Converts a string of list to dictionary
    l.append(i['name'])
  return l

In [ ]:
movie['genres'] = movie['genres'].apply(convert)

<ipython-input-47-fd033edce029>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['genres'] = movie['genres'].apply(convert)


In [ ]:
movie['keywords'] = movie['keywords'].apply(convert)

<ipython-input-48-3731033d75e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['keywords'] = movie['keywords'].apply(convert)


In [ ]:
def convert3(obj):
  l = []
  count = 0
  for i in ast.literal_eval(obj):
    if count != 3:
      l.append(i['name'])
      count += 1
    else:
      break
  return l

In [ ]:
movie['cast'] = movie['cast'].apply(convert3)

<ipython-input-50-cdae6cf1cff5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['cast'] = movie['cast'].apply(convert3)


In [ ]:
def director(obj):
  l = []
  count = 0
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      l.append(i['name'])
      break
  return l

In [ ]:
movie['crew'] = movie['crew'].apply(director)

<ipython-input-52-55fbe3c5cd1b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['crew'] = movie['crew'].apply(director)


In [ ]:
movie['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [ ]:
movie['overview'] = movie['overview'].apply(lambda x: str(x).split())

<ipython-input-54-a7168805ee9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['overview'] = movie['overview'].apply(lambda x: str(x).split())


In [ ]:
movie['genres'][0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [ ]:
movie['genres'] = movie['genres'].apply(lambda x:[i.replace(" ","") for i in x])

<ipython-input-56-858e464213ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['genres'] = movie['genres'].apply(lambda x:[i.replace(" ","") for i in x])


In [ ]:
movie['genres'][0]

['Action', 'Adventure', 'Fantasy', 'ScienceFiction']

In [ ]:
movie['crew'] = movie['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movie['keywords'] = movie['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movie['cast'] = movie['cast'].apply(lambda x:[i.replace(" ","") for i in x])

<ipython-input-58-30d353ce8f3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['crew'] = movie['crew'].apply(lambda x:[i.replace(" ","") for i in x])
<ipython-input-58-30d353ce8f3a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['keywords'] = movie['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
<ipython-input-58-30d353ce8f3a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [ ]:
movie.head(1)

,genres,keywords,title,overview,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [ ]:
movie['tags'] = movie['keywords'] + movie['cast'] + movie['crew'] + movie['genres'] + movie['overview']

<ipython-input-60-d3eb5870d035>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['tags'] = movie['keywords'] + movie['cast'] + movie['crew'] + movie['genres'] + movie['overview']


In [ ]:
FINAL_DF = movie[['movie_id','title','tags']]

In [ ]:
FINAL_DF['tags'] = FINAL_DF['tags'].apply(lambda x:" ".join(x))

<ipython-input-62-984c22840993>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINAL_DF['tags'] = FINAL_DF['tags'].apply(lambda x:" ".join(x))


In [ ]:
FINAL_DF['tags'] = FINAL_DF['tags'].apply(lambda x:x.lower())

<ipython-input-63-04196d6826df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINAL_DF['tags'] = FINAL_DF['tags'].apply(lambda x:x.lower())


**DATAFRAME WITH EXTRACTED FEATURES(PREPROCESSED FEATURES)**

In [ ]:
FINAL_DF

,movie_id,title,tags
0,19995,Avatar,cultureclash future spacewar spacecolony socie...
1,285,Pirates of the Caribbean: At World's End,ocean drugabuse exoticisland eastindiatradingc...
2,206647,Spectre,spy basedonnovel secretagent sequel mi6 britis...
3,49026,The Dark Knight Rises,dccomics crimefighter terrorist secretidentity...
4,49529,John Carter,basedonnovel mars medallion spacetravel prince...
...,...,...,...
4804,9367,El Mariachi,unitedstates–mexicobarrier legs arms paperknif...
4805,72766,Newlyweds,edwardburns kerrybishé marshadietlein edwardbu...
4806,231617,"Signed, Sealed, Delivered",date loveatfirstsight narration investigation ...
4807,126186,Shanghai Calling,danielhenney elizacoupe billpaxton danielhsia ...


In [ ]:
import nltk

**STEMMING:      **
To convert the extracted Features which are words to their base form

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
  y = []

  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [ ]:
FINAL_DF['tags'] = FINAL_DF['tags'].apply(stem)

<ipython-input-68-1e83dc734b13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINAL_DF['tags'] = FINAL_DF['tags'].apply(stem)


**Count vectoriser**
To convert the sentence into vector with 5000 dimensions (most frequent words)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english')

In [ ]:
vectors = cv.fit_transform(FINAL_DF['tags']).toarray()

In [ ]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

consine similarity between the matrices

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matix = cosine_similarity(vectors)

In [ ]:
similarity_matix[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [ ]:
def recomend(movie):
  index = FINAL_DF[FINAL_DF['title'] == movie].index[0]
  print(index)
  recommeded_list = sorted(enumerate(similarity_matix[index]),reverse =True,key = lambda x : x[1])[1:6]
  for i in recommeded_list:
    print(FINAL_DF.iloc[i[0]])


In [ ]:
sorted(enumerate(similarity_matix[0]),reverse =True,key = lambda x : x[1])[1:6]

[(1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574)]

In [ ]:
recomend('Avatar')

0
movie_id                                                  440
title                             Aliens vs Predator: Requiem
tags        predat nationalguard hybrid alien morgu alienp...
Name: 1216, dtype: object
movie_id                                                  679
title                                                  Aliens
tags        android extraterrestrialtechnolog spacemarin s...
Name: 2409, dtype: object
movie_id                                               270938
title                                           Falcon Rising
tags        yakuza marin fight michaeljaiwhit nealmcdonoug...
Name: 3731, dtype: object
movie_id                                                  602
title                                        Independence Day
tags        spacecraft patriot countdown independ invas wa...
Name: 507, dtype: object
movie_id                                                 7450
title                                              Titan A.E.
tags        monster galaxi 

In [ ]:
import pickle

In [ ]:
pickle.dump(FINAL_DF.to_dict(),open('MOVIE_RECOMENDATION.pkl','wb'))

In [ ]:
pickle.dump(similarity_matix,open('MOVIE_RECOMENDATION(model).pkl','wb'))